In [14]:
from google.cloud import bigquery, storage
import fastavro


In [15]:
bq_project = "evident-data-dev"

backup_project = "evident-data-dev-backups"
backup_bucket = "lucas-test-exports"

full_table_name = "N/A"

try:

    bq_client = bigquery.Client(project=bq_project)
    datasets = bq_client.list_datasets()

    job_config = bigquery.ExtractJobConfig()
    job_config.destination_format = bigquery.DestinationFormat.AVRO
    job_config.use_avro_logical_types = True

    for dataset in datasets: 

        tables = bq_client.list_tables(dataset.dataset_id)
        for table in tables:

            full_table_name = f"{dataset.dataset_id}.{table.table_id}"
            gcs_uri = f"gs://{backup_bucket}/{backup_project}/{dataset.dataset_id}/{table.table_id}-*.avro"
            
            extract_job = bq_client.extract_table(
            table,
            gcs_uri,
            job_config=job_config
        )   

except Exception as e:
    error_msg = f"Error backing up table {full_table_name}: {str(e)}"


